# Contents
Thank you for viewing this notebook.
Using WM-811K dataset, I will try image retrieval task. This notebook aims to implement the code with `PyTorch Metric Learning` library. This offers us the easiest way to build metric learning model and image search methods. 

You will find the simple and effective implementation!

### Preprocess(minimum)
WM-811K dataset containes the number of 811K data. However, there are only 25K data with specified labels. I will extract these with some other prerprocessing as well.

### Model, Dataset class definition
- Model: The most simple convnet with triplet loss
- Dataset: It returns X(torch.Tensor) and y(int). Yes, this is a kind of supervised learning.

### Train
With `trainer` API, I simply executed model training in metric learning way.

### Inference
- image retrieval with nearest neighbor search

### Some experiments

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wm811k-wafer-map/LSWMD.pkl


It's ok when you see the message,`/kaggle/input/wm811k-wafer-map/LSWMD.pkl`.  

上記実行後、`/kaggle/input/wm811k-wafer-map/LSWMD.pkl`と表記が出ればok

# Preprocessing, 前処理

In [2]:
import os
import warnings
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
print(os.listdir("../input"))
warnings.filterwarnings("ignore")

['wm811k-wafer-map']


In [3]:
# It takes around 2 minutes on Kaggle notebook.
df = pd.read_pickle("../input/wm811k-wafer-map/LSWMD.pkl")
print(df.info())
display(df.head(3))

KeyboardInterrupt: 

In [ ]:
# label extraction  e.g.)[Loc] → Loc　 @failureType, trianTestLabel column
f_squeeze = lambda x: str(np.squeeze(x))
df["failureType"] = df["failureType"].map(f_squeeze)
df["trianTestLabel"] = df["trianTestLabel"].map(f_squeeze)
df.head()

In [ ]:
# drop the row data with no specified label
eliminate_list = ['none', '[]']
df_with_label = df.query(f"failureType not in {eliminate_list}")
print(f"{len(df_with_label)}")
print("Label list: ", df_with_label["failureType"].unique())
df_with_label.head(3)

In [ ]:
class2idx = {
    'Loc': 0,
    'Edge-Loc': 1,
    'Center': 2,
    'Edge-Ring': 3, 
    'Scratch': 4,
    'Random': 5, 
    'Near-full': 6,
    'Donut': 7
}
idx2class = {v: k for k, v in class2idx.items()}

# The following needs only when you want to save the dict above. Not always necessary.
class2idx_json_path = "./config_class2idx.json"
with open(class2idx_json_path, "w") as f:
    json.dump(class2idx, f)
# ↑保存終了

# New column → encoded_labels
df_with_label["encoded_labels"] = df_with_label["failureType"].replace(class2idx)

print(df_with_label["encoded_labels"].value_counts())
df_with_label.reset_index(inplace=True, drop=True)
df_with_label.to_pickle("./LSWMD_25519.pkl")  # ラベル有データ。ラベルエンコーディング等の前処理終わりデータになった。
df_with_label.head(3)

In [ ]:
train_data_length = int(0.8*len(df_with_label))  # training data for 80% of all data
df_with_label = df_with_label.sample(frac=1, random_state=2)
df_train, df_test = df_with_label[:train_data_length], df_with_label[train_data_length:]
df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

train_data_length = int(0.7*len(df_train))  # Again, split the train data: train ⇨ train & val
df_train, df_val = df_train[:train_data_length], df_train[train_data_length:]
print(f"training: {len(df_train)}, val: {len(df_val)}, test: {len(df_test)}, (all: {len(df_with_label)})")

# This aims to save the data into your environment.
df_train.to_pickle("dataset_train.pickle")
df_val.to_pickle("dataset_val.pickle")
df_test.to_pickle("dataset_test.pickle")
print(os.listdir())

In [ ]:
df_train.head(10)

In [ ]:
idx2class = {v: k for k, v in class2idx.items()}
batch_size = 32
plt.figure(figsize=(12,12))
for i in range(batch_size):
    ax = plt.subplot(4, 8, i+1)
    ax.axis("off")
    ax.imshow(df_train["waferMap"][i])
    ax.set_title(f'{idx2class[df_train["encoded_labels"][i]]}:{str(df_train["encoded_labels"][i])}', fontsize=8)

Preprocess ended! 前処理終了
# Model definition、Training
- Simple model definition 
- Implement with `pytorch-metric-learning` library

In [ ]:
!pip install pytorch-metric-learning==1.6.2
# !pip install -q pytorch-metric-learning[with-hooks]
!pip install -q scanpy
!pip install faiss-cpu

In [ ]:
from typing import Any, List
import torch
import torchvision
from torch import optim 
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
from pytorch_metric_learning import losses, miners, distances, reducers, samplers
from pytorch_metric_learning import trainers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.utils import logging_presets
import cv2
import umap

In [ ]:
# Dataset class definition
class WafermapTripletDataset(Dataset):
    def __init__(self, 
                 dataset_path: str = "",
                 column_name_apply_wafermap = "waferMap",
                 column_name_label: str = "encoded_labels",
                 resized_image_size: int = 80,
                 transforms: Any = None,
                 phase: str = "train"):
        super().__init__()
        self.__dict__.update(locals())  # 個人的にはあまり好きな書き方ではない。1つずつ定義を書いてもいいと思う。
        self._init_dataset()
        self.transforms = transforms
        self.phase = phase 
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        apply_wafermap, label = self.apply_wafer_maps[index], self.labels[index]
        if self.transforms:
            apply_wafermap = self.transforms(apply_wafermap, self.phase)
        else:
            pass
        apply_wafermap = apply_wafermap.to(torch.float32)

        return (apply_wafermap, label)

    def _init_dataset(self):
        self.df = pd.read_pickle(self.dataset_path).reset_index()
        self.labels = self.df[self.column_name_label]

        # modify wafermaps
        # apply image
        self.df[self.column_name_apply_wafermap] = self.df[self.column_name_apply_wafermap].apply(lambda x: cv2.resize(x, (self.resized_image_size, self.resized_image_size)))
        self.df[self.column_name_apply_wafermap] = self.df[self.column_name_apply_wafermap].apply(lambda x: np.repeat(x[..., np.newaxis], 1, -1))
        self.apply_wafer_maps = self.df[self.column_name_apply_wafermap].apply(lambda x: x.transpose((2,1,0)))  # inputの(バッチ、チャネル、たて、よこ)順番調整
        self.apply_wafer_maps = self.apply_wafer_maps + 1e-6
        self.apply_wafer_maps = self.apply_wafer_maps.apply(lambda x: torch.from_numpy(x))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.25)
        self.fc = nn.Linear(18432, 1152)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = F.relu(x)
        x = self.dropout2(x)
        return x


class Embedder(nn.Module):
    def __init__(self):
        super(Embedder, self).__init__()
        self.fc = nn.Linear(1152, 128)

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model
trunk = Net()
trunk = torch.nn.DataParallel(trunk.to(device))
embedder = Embedder()
embedder = torch.nn.DataParallel(embedder.to(device))
models = {"trunk": trunk, "embedder": embedder} 

# optimizer setting
trunk_optimizer = optim.Adam(trunk.parameters(), lr=0.01)
embedder_optimizer = optim.Adam(embedder.parameters(), lr=0.005)
optimizers = {"trunk_optimizer": trunk_optimizer, 
              "embedder_optimizer": embedder_optimizer}

distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0) # 1対のペアごとにロスが計算されるが、揃ったものすべてを使うのではなく一定の閾値以上のものだけ使おうという処理。組み合わせ多いからかな。
loss = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
miner = miners.TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="semihard")  # ペアのうち、semihardとなるものを探す(=mining)機能。ここも調整の余地あり
loss_funcs = {"metric_loss": loss}
mining_funcs = {"tuple_miner": miner}

# visual_hook implementation for result visualization
record_keeper, _, _ = logging_presets.get_record_keeper("logs", "tensorboard")  # ログ機能。logs, tensorboardフォルダを作って色々格納してくれるが本筋に影響なし
print(f"{record_keeper}")
hooks = logging_presets.get_hook_container(record_keeper)  # 学習イテレーションごとに追加処理を行うときに使う（embeddingをプロットしたい、）

def visualizer_hook(umapper, umap_embeddings, labels, split_name, keyname, epoch):
    class_labels = np.unique(labels)
    num_classes = len(class_labels)

    fig = plt.figure(figsize=(8, 6))
    colors = [plt.cm.nipy_spectral(i) for i in np.linspace(0, 0.9, num_classes)]
    plt.gca().set_prop_cycle(cycler("color", colors))

    for i, lab in enumerate(class_labels):
        idx = labels == class_labels[i]
        plt.plot(umap_embeddings[idx, 0], umap_embeddings[idx, 1], ".", markersize=3, label=lab) 

    plt.legend(frameon=False, fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    os.makedirs("result", exist_ok=True)
    plt.savefig(f"result/{epoch:02d}.png")
    plt.show()
    plt.close()

In [ ]:
dataset_train_path = "./dataset_train.pickle"
dataset_val_path = "./dataset_train.pickle"
dataset_test_path = "./dataset_test.pickle"
train_dataset = WafermapTripletDataset(dataset_train_path, resized_image_size=28, column_name_apply_wafermap="waferMap")
val_dataset = WafermapTripletDataset(dataset_val_path, resized_image_size=28, column_name_apply_wafermap="waferMap")
test_dataset = WafermapTripletDataset(dataset_test_path, resized_image_size=28, column_name_apply_wafermap="waferMap")

In [ ]:
# Tester definition
tester = testers.GlobalEmbeddingSpaceTester(end_of_testing_hook=hooks.end_of_testing_hook, 
                                            visualizer=umap.UMAP(), 
                                            visualizer_hook=visualizer_hook,
                                            dataloader_num_workers=4)

# Hook setting　→　1 epochごとにやってくれる処理をまとめるモジュールと考えればOK
dataset_dict = {"val": val_dataset}
model_dir = "./saved_models"
end_of_epoch_hook = hooks.end_of_epoch_hook(tester, 
                                            dataset_dict, 
                                            model_dir, 
#                                             test_interval=1,   # depending on the PML version, this might be an error
#                                             patience=1,  # depending on the PML version, this might be an error
                                           )

# Model training
num_epochs = 50
batch_size = 48

# PML training API：trainer　⇒　モデル、ロス、最適化関数、データセットクラス、マイニング関数、等を与える
trainer = trainers.MetricLossOnly(models,
                                  optimizers,
                                  batch_size,
                                  loss_funcs,
                                  mining_funcs,
                                  train_dataset,
                                #   sampler=sampler,
                                  dataloader_num_workers=4,
                                  end_of_iteration_hook=hooks.end_of_iteration_hook,
                                  end_of_epoch_hook=end_of_epoch_hook)
trainer.train(num_epochs=num_epochs)

In [ ]:
# you can plot learning history, but it seems to depend on PML version. I am trying.

# loss_history = hooks.get_loss_history()
# plt.plot(loss_history["metric_loss"], "r", alpha=0.6, label="loss")
# plt.legend()

# Inference(Image Retrieval) 推論（画像検索）
- Load the model
- `InferenceModel`(pytorch-metric-learning module) ⇨ executes nearest neighbor search

In [ ]:
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.utils import common_functions as c_f
from pytorch_metric_learning.utils.inference import InferenceModel, MatchFinder


# saved_trunk_path = "./saved_models/trunk_best1.pth"
# saved_embedder_path = "./saved_models/embedder_best1.pth"

# trunk = Net()
# trunk.load_state_dict(torch.load(saved_trunk_path))
# trunk.cpu()

# embedder = Embedder()
# embedder.load_state_dict(torch.load(saved_embedder_path))
# embedder.cpu()

match_finder = MatchFinder(distance=CosineSimilarity(), threshold=0.7)
inference_model = InferenceModel(trunk=trunk,
                                 embedder=embedder,
                                 match_finder=match_finder,
                                 data_device="cpu",
                                 )

In [ ]:
# knn!(knn way nearest neighbor search)： This makes indices of the dataset
inference_model.train_knn(test_dataset)

In [ ]:
# データセットクラスで定義したラベル（0－7）それぞれに対応するインデックスを取得できる
# PML offeres some utils function. You can get the data separated by labels.
labels_to_indices = c_f.get_labels_to_indices(test_dataset.labels)
# 試しにラベル3、ラベル６のインデックス集合をそれぞれ取得する
# For example, let's get the data with label 3 & 6
classA, classB = labels_to_indices[3], labels_to_indices[6]
img_type = classA   # ラベル3に対応するインデックスの集合(Data with label '3')

# 検索実行 Let's retrieve!
img, label = test_dataset[img_type[2]][0].unsqueeze(0), test_dataset[img_type[2]][1]
print(type(img), img.shape)

top_k = 10
distances, indices = inference_model.get_nearest_neighbors(img, k=top_k)
distances_list = list(distances.cpu().numpy()[0])
indices_list = list(indices.cpu().numpy()[0])

In [ ]:
print(f"top'{top_k}' index: ", indices_list)
df_result = test_dataset.df.iloc[indices_list, :]
df_result.head(10)

# Experiment Results, 実験結果
先ほどやったことを視覚化しながら再度実行してみる
- Comparison of query image and retrieved ones
- Visualize the embedding space(On kaggle notebook, it doesn't work. You can download it and try on the other environment)

In [ ]:
# クエリ情報の抜出 (ラベル１のデータを抜き出し、とりあえず最初の行にあるものをクエリとしてみる)
encoded_label = 4  # 自由に変えてトライ, Let's change.
df_tmp = test_dataset.df.query(f"encoded_labels=={encoded_label}").reset_index()
query, query_label = df_tmp["waferMap"][0], df_tmp["encoded_labels"][0]
print("Before preprocess: ", type(query), query.shape) # この時点ではnumpy.ndarrayになっている
plt.imshow(query)
plt.title(f"Query image: {idx2class[query_label]}")


# クエリ画像をモデルに入力するためのtorch用前処理
query = query.transpose(2,0,1) # shape:(a,b,c) → (c,a,b)
query = torch.from_numpy(query).float()
query = query.unsqueeze(0) # shape:(a,b,c) → (1,a,b,c) のように1を追加。PyTorchのお作法（バッチ次元）
print("After preprocess: ", type(query), query.shape) # この時点ではtorch.Tensorになっている
assert isinstance(query, torch.Tensor)

# Search! 実験（検索実行！）
top_k = 10
distances, indices = inference_model.get_nearest_neighbors(query, k=top_k)
distances_list = list(distances.cpu().numpy()[0])
indices_list = list(indices.cpu().numpy()[0])

print(f"top'{top_k}' index: ", indices_list)
df_result = test_dataset.df.iloc[indices_list, :].reset_index(drop=True)


# Visualize results. 結果表示
display(df_result.head(10))
plt.figure(figsize=(16,16))
for i in range(top_k):
    ax = plt.subplot(8, 6, i+1)
    ax.axis("off")
    ax.imshow(df_result["waferMap"][i])
    ax.set_title(f'{idx2class[df_result["encoded_labels"][i]]}-{str(df_result["encoded_labels"][i])}', 
                 fontsize=8,
                )
plt.tight_layout()
plt.show()

# 以上,結果のまとめと今後とるべき対策
- クエリに対して画像の検索を実施した
- クエリに対して検索上位に出てきたものは似てるものだけではなかった。
    * 学習器を変える
    * 画像毎のサイズ違いに注意する
    * augmentationする

## 発展的知識
- プロダクション上では一般に検索DBが大きくなりやすい（桁違いに大きくなる）
- クエリの特徴量抽出は特徴抽出器のサイズ依存だが、検索は「検索手法依存」となる
- 検索には一定の高速性が求められるため、「検索手法」の工夫が世の中にはたくさんある
    * 全探索手法
    * 最近傍探索手法
    * 近似最近傍探索手法
    * 様々な後処理手法（検索結果を出したのち、その中で再度何かしら計算を行い精度を高める）
- 画像検索は「クエリと似たものは何か？」を探すタスクであり、学習時にないデータを推論する（=検索する）ことに使う
- metric learningは学習データの**相対的な位置関係**をロス関数にして学習しており、顔認証や画像検索タスクに用いられることで知られる（世の中のどのシステムに具体的に導入されてるかは把握してません）
- 画像検索コンペの最上位でもmetric learningは未だに使用されており、工夫はいるが強力な手法である

# 以下、ちょっとわからないこと出たのでやりません。機能だけ紹介します。

再度掲載- クラスと対応番号
```
class2idx = {
    'Loc': 0,
    'Edge-Loc': 1,
    'Center': 2,
    'Edge-Ring': 3, 
    'Scratch': 4,
    'Random': 5, 
    'Near-full': 6,
    'Donut': 7
}
```

In [ ]:
# データセットクラスで定義したラベル（0－7）それぞれに対応するインデックスを取得できる
labels_to_indices = c_f.get_labels_to_indices(train_dataset.labels)
# 試しにラベル１、ラベル６のインデックス集合をそれぞれ取得する
classA, classB = labels_to_indices[3], labels_to_indices[9]
img_type = classA   # ラベル3に対応するインデックスの集合

# 検索実行
img, label = test_dataset[img_type[2]][0].unsqueeze(0), test_dataset[img_type[2]][1]
print(type(img), img.shape)
# distances, indices = inference_model.get_nearest_neighbors(img, k=10)

print(classA)

In [ ]:
print(indices.cpu().numpy())
print(indices.cpu().numpy()[0].shape)

In [ ]:
def print_decision(is_match):
    if is_match:
        print("Same class")
    else:
        print("Different class")

# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

# inv_normalize = transforms.Normalize(
#     mean=[-m / s for m, s in zip(mean, std)], std=[1 / s for s in std]
# )
def imshow(img, title=None, figsize=(8, 4)):
    # img = inv_normalize(img)
    npimg = img.numpy()
    plt.figure(figsize=figsize)
    plt.imshow(np.transpose(npimg, (1, 2, 0))[:,:,0])
    plt.title(title)
    plt.show()

# compare two images of the same class
(x, _), (y, _) = test_dataset[classA[0]], test_dataset[classA[1]]
print(x.shape, y.shape)
decision = inference_model.is_match(x.unsqueeze(0), y.unsqueeze(0))
stack_image = torch.stack([x, y], dim=0)  # double torch.Size([20, 1, 28, 28]) -> torch.Size([2, 20, 1, 28, 28])
imshow(torchvision.utils.make_grid(stack_image), title=decision)
print_decision(decision)